In [1]:
!pip install -U spacy -q

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.1.123+-x86_64-with-glibc2.35
Python version   3.12.11                       
Pipelines        en_core_web_sm (3.8.0)        



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
import json
f = open('training_data.json')
TRAIN_DATA = json.load(f)

In [6]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in annot["entities"]:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
        print("Skipping entity")
    else:
        ents.append(span)
  doc.ents = ents
  db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object


100%|██████████| 15/15 [00:00<00:00, 1106.07it/s]


In [7]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [8]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     61.56    0.00    0.00    0.00    0.00
 66     200        655.21   3599.74   88.57   87.32   89.86    0.89
133     400        114.87   1162.49   88.57   87.32   89.86    0.89
200     600        142.64   1100.17   88.57   87.32   89.86    0.89
289     800        188.02   1354.68   88.41   88.41   88.41    0.88
383    1000        152.47   1339.92   88.57   87.32   89.86    0.89
461    1200         93.74   1091.02   90.65   90.00   91.30    0.91
553    1400         94.91   1027.94   90.65   90.00   91.30    0.91
677    1600        161.52   1367.02 

In [9]:
nlp = spacy.load('/content/model-best')

In [10]:
doc = nlp('''From the Sensex firms, Sun Pharmaceutical, Tata Steel, Adani Ports, ICICI Bank, Bharti Airtel, Power Grid, Bharat Electronics Ltd, HDFC Bank, NTPC and Tata Motors were the laggards.

Hindustan Unilever and Tata Consultancy Services were the only gainers.

On Monday (August 25, 2025), the United States issued a draft order implementing an additional 25% tariff on Indian imports, which President Donald Trump had announced earlier, beginning August 27.According to the draft notice released by the Department of Homeland Security, the additional tariffs will cover Indian products "that are entered for consumption, or withdrawn from warehouse for consumption, on or after 12:01 am Eastern Daylight Time on August 27, 2025".

The notice explained that the higher levies were linked to "threats to the U.S. by the government of the Russian Federation," with India targeted in line with that strategy.

"The biggest headwind for Indian markets remains whether Nifty can scale the 'Wall of Worry' around the 50% Mr. Trump tariff set to kick in on August 27, which threatens to make almost all of India's $86.5 billion exports to the U.S. commercially unviable," Prashanth Tapse, Senior VP (Research) at Mehta Equities Ltd, said.''')

In [11]:
spacy.displacy.render(doc, style='ent', jupyter=True)